In [1]:
import os
import pandas as pd
import json

In [2]:
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())

In [3]:
herectvo = pd.read_json("herectvo_raw.json")

In [4]:
herectvo.shape

(86844, 12)

## Jen relevantní filmy

In [5]:
df = pd.read_json(os.path.join("data", "filmy.json"))

In [6]:
relevantni_filmy = df["Film"].to_list()

In [7]:
herectvo = herectvo[herectvo["Film"].isin(relevantni_filmy)]

In [8]:
herectvo.shape

(86651, 12)

## Opravy

In [9]:
for key, val in opravy["data_narozeni"].items():
    herectvo.loc[herectvo["Jméno"] == key, "Datum narození"] = val
for key, val in opravy["jmena"].items():
    herectvo.loc[herectvo["Jméno"] == key, "Jméno"] = val

## Doplnění dat z Wikidat

In [10]:
herectvo[herectvo["Jméno"] == "Jiří Maryško (81150)"].sample(1)

,Jméno,Bio,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Titul,Film,Rok,Úloha,Role,Alternativní jména
76674,Jiří Maryško (81150),None,None,None,None,None,Modrý tygr,Modrý tygr (66111),2012,Hrají,lepič plakátů,None


In [11]:
herectvo[herectvo["Jméno"] == "Jitka Sedláčková (1435)"].sample(1)

,Jméno,Bio,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Titul,Film,Rok,Úloha,Role,Alternativní jména
28231,Jitka Sedláčková (1435),None,None,None,None,None,Jedině Tereza,Jedině Tereza (402152),2021,Hrají,recepční Alena,None


In [12]:
with open(os.path.join("data_fixes", "data_z_wikidat.json"), encoding="utf-8") as data_z_wikidat:
    data_z_wikidat = json.loads(data_z_wikidat.read())

In [13]:
for key, val in data_z_wikidat.items():
    herectvo.loc[herectvo["Jméno"] == key, "Datum narození"] = val[0]

In [14]:
for key, val in data_z_wikidat.items():
    herectvo.loc[herectvo["Jméno"] == key, "Datum úmrtí"] = val[1]

In [15]:
herectvo[herectvo["Jméno"] == "Jitka Sedláčková (1435)"].sample(1)

,Jméno,Bio,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Titul,Film,Rok,Úloha,Role,Alternativní jména
28243,Jitka Sedláčková (1435),None,18.03.1961,None,None,None,"Mach, Šebestová a kouzelné sluchátko","Mach, Šebestová a kouzelné sluchátko (1659)",2001,Hrají,školníková,None


In [16]:
herectvo[herectvo["Jméno"] == "Jiří Maryško (81150)"].sample(1)

,Jméno,Bio,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Titul,Film,Rok,Úloha,Role,Alternativní jména
76663,Jiří Maryško (81150),None,04.04.1980,None,None,None,Tajemství staré bambitky 2,Tajemství staré bambitky 2 (402174),2021,Hrají,"princ Oleandr, nápadník",None


## Data a místa

In [17]:
def rozdel_misto(x):
    if type(x) == str:
        try:
            mesto = x.split(",")[0]
            stat = x.split(",")[1]
        except:
            mesto = None
            stat = x
    else:
        mesto = None
        stat = None
    return [mesto, stat]

In [18]:
herectvo["Město narození"] = herectvo["Místo narození"].apply(lambda x: rozdel_misto(x)[0])
herectvo["Stát narození"] = herectvo["Místo narození"].apply(lambda x: rozdel_misto(x)[1])
herectvo["Město úmrtí"] = herectvo["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
herectvo["Stát úmrtí"] = herectvo["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])

In [19]:
herectvo["Rok narození"] = herectvo["Datum narození"].astype(str).apply(lambda x: x[-4:])
herectvo["Rok narození"] = pd.to_numeric(herectvo["Rok narození"], errors='coerce') 

In [20]:
herectvo["Datum narození"] = pd.to_datetime(herectvo["Datum narození"], errors="ignore", format = "%d.%m.%Y")

In [21]:
herectvo["Rok úmrtí"] = herectvo["Datum úmrtí"].astype(str).apply(lambda x: x[-4:])
herectvo["Rok úmrtí"] = pd.to_numeric(herectvo["Rok úmrtí"], errors='coerce') 

In [22]:
herectvo["Datum úmrtí"] = pd.to_datetime(herectvo["Datum úmrtí"], errors="ignore", format = "%d.%m.%Y")

In [23]:
herectvo["Věk"] = herectvo["Rok"].astype(float) - herectvo["Rok narození"].astype(float)

## Uložení

In [24]:
herectvo.to_json(os.path.join("data", "herectvo.json"), orient="records")

In [25]:
import zipfile
try:
    import zlib
    mode= zipfile.ZIP_DEFLATED
except:
    mode= zipfile.ZIP_STORED

In [26]:
with zipfile.ZipFile(os.path.join("data", "herectvo.zip"), mode="w") as my_zip:
    my_zip.write(os.path.join("data", "herectvo.json"), compress_type=zipfile.ZIP_DEFLATED)